# Solving shortest path problem using gurobipy

In [1]:
from gurobipy import * 

In [3]:
nodes = ['1', '2', '3', '4']
links = [('1', '2'), ('1', '3'), ('2', '3'), ('2', '4'), ('3', '4')]
FS = {'1': ['2', '3'], '2':['3', '4'], '3':['4'], '4':[]}
BS = {'1':[], '2':['1'], '3':['1', '2'], '4':['2', '3']}
cost = {('1', '2'): 3, ('1', '3'): 7, ('2', '3'):1, ('2', '4'):3, ('3', '4'):1}

In [24]:
# Step 1: Intialize the model
m = Model('shortest path')# argument takes the name of the model.

# Step 2: Add decision variables
x = {}
for l in links:
    # For arguments refer to https://www.gurobi.com/documentation/current/refman/py_model_addvar.html
    x[l] = m.addVar(vtype = GRB.BINARY, name = str(l)) 
    

# Step 3: Define constraints
for i in nodes:
    diff_flow = sum([x[i, j] for j in FS[i]]) - sum([x[j, i] for j in BS[i]])
    if i == '1': # origin
        #For more details about addConstr refer to https://www.gurobi.com/documentation/current/refman/py_model_addconstr.html
        m.addConstr(diff_flow == 1)
    elif i == '4': # destination
        m.addConstr(diff_flow == -1)
    else:
        m.addConstr(diff_flow == 0) # other nodes

# Step 4: Prepare the objective 
obj = sum([cost[i, j]*x[i, j] for (i, j) in links])
m.setObjective(obj, sense =GRB.MINIMIZE) # For more details, refer to https://www.gurobi.com/documentation/current/refman/cs_model_setobjective.html

In [25]:
m.optimize() # Solve the model

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 5 columns and 10 nonzeros
Model fingerprint: 0x891a69c1
Variable types: 0 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%


In [29]:
if m.status == GRB.OPTIMAL:
    print(m.objVal)
    for l in links:
        if x[l].x > 0.2: # To get the value of the decision variable, you need to put "dot" followed by the decision variable.
            print(l, x[l].x)

5.0
('1', '2') 1.0
('2', '3') 1.0
('3', '4') 1.0
